In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

2025-06-10 12:24:52.178376: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-10 12:24:52.178794: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-10 12:24:52.181221: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-10 12:24:52.187685: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749522292.199087   10704 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749522292.20

In [2]:
# Step 2: Simulate the Dataset
num_datasets = 10**6  
sample_size = 100    
n_per_dist = num_datasets // 2

In [3]:
import random
seed = 12345
np.random.seed(seed)
random.seed(seed)

# Generate datasets
normal_samples_m0 = np.random.normal(loc=0, scale=1, size=(n_per_dist, sample_size))
normal_samples_m1_1 = np.random.normal(loc=0.1, scale=1, size=(n_per_dist, sample_size))

# Combine the datasets
normal_samples = np.vstack([normal_samples_m0, normal_samples_m1_1])

In [4]:
del normal_samples_m0, normal_samples_m1_1

In [5]:
# Combine all datasets
X = normal_samples
# Normalize the data
X = (X - np.mean(X, axis=1, keepdims=True)) / np.std(X, axis=1, keepdims=True)
del normal_samples

In [6]:
X.shape

(1000000, 100)

In [7]:
# Creation of labels for model selection

mu0_labels = np.zeros((n_per_dist, 1))
mu1_1_labels = np.ones((n_per_dist, 1))       

# Combine all labels
y_class = np.vstack([mu0_labels, mu1_1_labels])

y_class = to_categorical(y_class, num_classes=2)

del mu0_labels, mu1_1_labels

In [8]:
# Creation of parameter labels

y_params_mu0 = np.zeros((n_per_dist, 1))
y_params_mu1_1 = np.full((n_per_dist, 1), 0.1)

y_params = np.vstack([y_params_mu0, y_params_mu1_1])

y_params.shape

del y_params_mu0, y_params_mu1_1

In [9]:
### Step 3: Split the Data into Training and Testing Sets
X_train, X_test, y_class_train, y_class_test, y_params_train, y_params_test = train_test_split(
    X, y_class, y_params, test_size=0.2, random_state=42)

In [10]:
### Step 4: Define the Multi-Task Model
### Create a model with a shared base and two task-specific output layers: 
# one for classification and another for parameter estimation.

# Input layer
input_layer = Input(shape=(sample_size,))

# Shared base layers
shared = layers.Dense(128, activation='relu')(input_layer)
shared = layers.Dense(64, activation='relu')(shared)

# Classification output
class_output = layers.Dense(2, activation='softmax', name='class_output')(shared)

# Parameter estimation output
param_output = layers.Dense(2, activation='linear', name='param_output')(shared)

# Define the model
model = models.Model(inputs=input_layer, outputs=[class_output, param_output])

2025-06-10 12:24:57.458439: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [11]:
### Step 5: Compile the Model
model.compile(optimizer='adam',
              loss={'class_output': 'categorical_crossentropy', 'param_output': 'mse'},
              loss_weights={'class_output': 1.0, 'param_output': 0.5},
              metrics={'class_output': 'accuracy', 'param_output': 'mae'})

In [12]:
### Step 6: Train the Model
history = model.fit(X_train, {'class_output': y_class_train, 'param_output': y_params_train},
                    epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 24s 1ms/step - class_output_accuracy: 0.5010 - class_output_loss: 0.6951 - loss: 0.6992 - param_output_loss: 0.0082 - param_output_mae: 0.0637 - val_class_output_accuracy: 0.5001 - val_class_output_loss: 0.6937 - val_loss: 0.6949 - val_param_output_loss: 0.0025 - val_param_output_mae: 0.0500
Epoch 2/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step - class_output_accuracy: 0.4988 - class_output_loss: 0.6932 - loss: 0.6945 - param_output_loss: 0.0025 - param_output_mae: 0.0500 - val_class_output_accuracy: 0.5001 - val_class_output_loss: 0.6931 - val_loss: 0.6944 - val_param_output_loss: 0.0025 - val_param_output_mae: 0.0500
Epoch 3/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 24s 1ms/step - class_output_accuracy: 0.5007 - class_output_loss: 0.6932 - loss: 0.6944 - param_output_loss: 0.0025 - param_output_mae: 0.0500 - val_class_output_accuracy: 0.5000 - val_class_output_loss: 0.6931 - val_loss: 0.6944 - val_param_output_loss: 0.0025 - val_param_output_m

In [13]:
### Step 7: Evaluate the Model on the Test Dataset
results = model.evaluate(X_test, {'class_output': y_class_test, 'param_output': y_params_test})
print(f"Test Classification Accuracy: {results[3]:.4f}")
print(f"Test Parameter Estimation MAE: {results[4]:.4f}")

6250/6250 ━━━━━━━━━━━━━━━━━━━━ 6s 941us/step - class_output_accuracy: 0.4984 - class_output_loss: 0.6933 - loss: 0.6945 - param_output_loss: 0.0025 - param_output_mae: 0.0500
Test Classification Accuracy: 0.5000
Test Parameter Estimation MAE: 0.0500


In [14]:
# Step 7: Prediction 
# Parameters
sample_size = 100
n_test_per_model = 100

# Means for each model
mu_values = [0.0, 0.1]

# Generate test datasets
test_data = []
test_labels = []

for label, mu in enumerate(mu_values):
    data = np.random.normal(loc=mu, scale=1.0, size=(n_test_per_model, sample_size))
    test_data.append(data)
    test_labels.append(np.full((n_test_per_model,), label))

# Combine all into arrays
X_new = np.vstack(test_data)
y_class_new = np.concatenate(test_labels)

print("Test data shape:", X_new.shape)       
print("Test labels shape:", y_class_new.shape)  

Test data shape: (200, 100)
Test labels shape: (200,)


In [15]:
predictions = model.predict(X_new)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [16]:
probabilities = predictions[0]  # shape (n_samples, 6)
n_samples = probabilities.shape[0]
n_classes = probabilities.shape[1]

class_predictions_m0m1 = np.array([
    np.random.choice(n_classes, p=probabilities[i])
    for i in range(n_samples)
])

In [17]:
param_predictions_m0m1 = predictions[1]

In [18]:
from sklearn.metrics import confusion_matrix
cm_m0m1 = confusion_matrix(y_class_new, class_predictions_m0m1)
print(cm_m0m1)

[[50 50]
 [48 52]]


In [19]:
print(probabilities[:5])

[[0.49659434 0.5034057 ]
 [0.49659434 0.5034057 ]
 [0.49659434 0.5034057 ]
 [0.49659434 0.5034057 ]
 [0.49659434 0.5034057 ]]


In [20]:
import pandas as pd

df = pd.DataFrame(probabilities[:100, 0], columns=["NN"])

# Save to CSV
df.to_csv("normal_model0_NN_probabilities.csv", index=False)

In [21]:
params_new = predictions[1]
print("Estimated parameters shape:", params_new.shape)
print("First 5 parameter estimates:\n", params_new[:5])

Estimated parameters shape: (200, 2)
First 5 parameter estimates:
 [[0.0494449 0.0494449]
 [0.0494449 0.0494449]
 [0.0494449 0.0494449]
 [0.0494449 0.0494449]
 [0.0494449 0.0494449]]


In [22]:
df = pd.DataFrame(params_new[:100, 0], columns=["NN"])

# Save to CSV
df.to_csv("normal_model0_NN_params.csv", index=False)